In [1]:
import numpy as np


In [4]:
from keras import layers
from keras import models

Using TensorFlow backend.


In [24]:
def LeNet():
    model = models.Sequential()
    model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=[28,28,1]))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3),activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3),activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(512,activation='relu'))
    model.add(layers.Dense(10,activation='softmax'))
    model.summary()
    return model

In [8]:
from keras.datasets import mnist
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [9]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 52s - loss: 0.1463 - acc: 0.9536    - ETA: 1s 
Epoch 2/5
60000/60000 [==============================] - 53s - loss: 0.0440 - acc: 0.9859    
Epoch 3/5
60000/60000 [==============================] - 51s - loss: 0.0300 - acc: 0.9913    
Epoch 4/5
60000/60000 [==============================] - 56s - loss: 0.0246 - acc: 0.9927    
Epoch 5/5
60000/60000 [==============================] - 47s - loss: 0.0192 - acc: 0.9948    


In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 9952/10000 [============================>.] - ETA: 0s

In [13]:
print("test_acc: {}".format(test_acc))

test_acc: 0.9901


In [18]:
!ls

best_weight.h5	Untitled.ipynb


In [15]:
import os
curdir = os.getcwd()

In [16]:
#save model
model.save_weights(os.path.join(curdir,'best_weight.h5'))

In [17]:
#load model

In [25]:
model_load = LeNet()
model_load.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               295424    
__________

In [26]:
model_load.load_weights(os.path.join(curdir,'best_weight.h5'))
model_load.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               295424    
__________

In [27]:
model_load.pop() # remove output layer
model_load.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               295424    
Total para

In [28]:
embedding = model_load

In [78]:
(_, _), (test_images, test_labels) = mnist.load_data()
X =test_images
label = test_labels
print(" num of X:",X.shape)
print(" num of label:",label.shape)

 num of X: (10000, 28, 28)
 num of label: (10000,)


In [57]:
X = X.reshape((X.shape[0], 28, 28, 1)).astype('float32') / 255

In [58]:
predict_embedded = embedding.predict(X)

In [59]:
predict_embedded.shape

(10000, 512)

In [70]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
embs_2 = pca.fit_transform(predict_embedded)  # (70000, 128)

In [71]:
pca.explained_variance_ratio_

array([0.20523377, 0.18767802, 0.10292097], dtype=float32)

In [72]:
embs_2.shape

(10000, 3)

In [73]:
data = np.concatenate([embs_2,label.reshape(-1,1)],axis=1)
data.shape

(10000, 4)

In [74]:
import pandas as pd
dataset = pd.DataFrame({'X': data[:, 0], 'Y': data[:, 1],'Z':data[:,2],'Label':data[:,3]})

In [77]:
import plotly.express as px
fig = px.scatter_3d(dataset, x='X', y='Y',z="Z",
                color='Label')
fig.show()